In [2]:
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
from openai import OpenAI
import os
load_dotenv()
index_name = "bezzie-index"
pinecone = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [3]:
index_name = "bezzie-test-index"
def create_index():
    if not pinecone.has_index(index_name):
        pinecone.create_index(
            name=index_name,
            metric="cosine",
            dimension=1536,
            spec=ServerlessSpec(cloud="aws", region="us-east-1"),
            tags={
                "environment": "development"
            }
        )
    else:
        return pinecone.describe_index(name=index_name)
indexRes = create_index()

In [5]:
def createEmbedding():
    res = client.embeddings.create(
        model="text-embedding-3-small",
        input="The Person is little bit depressed and going through tough time in work and his social life including his family and friends that are avoiding him due to his constant failures, he is also going through a hard time due to no job interview calls",
        dimensions=1536,
        encoding_format='float'
    )
    
    return res
embd = createEmbedding()

In [15]:
for i in embd.data:
    print(i.embedding)

[-0.012533069, -0.005189569, 0.048936326, 0.04188266, -0.013924276, -0.047056977, 0.019391477, 0.03214421, 0.01629177, -0.023589505, -0.03060656, -0.042297583, -0.051840775, 0.010031337, 0.01574261, 0.03385271, 0.041687403, 0.015657183, -0.026457345, 0.034755774, 0.024309516, 0.05896766, 0.01735348, 0.0040973495, 0.027799739, 0.007438077, 0.003969212, 0.018097898, 0.07453942, 0.015461926, 0.036805972, -0.04256606, 0.005589236, 0.032363873, -0.028336694, 0.0015986679, 0.014131737, -0.0033620843, 0.009146578, 0.01055609, 0.027189558, -0.011892381, 0.011611699, 0.017255852, -0.007273329, -0.009469972, -0.016340584, -0.013228673, 0.039686017, -0.020270133, -0.030069603, 0.014046311, 0.032852016, 0.069853246, -0.019843008, -0.025310209, -0.0067790844, -0.05135263, -0.022491183, 0.006937731, 0.057600863, -0.032168616, 0.019891823, -0.020001655, -0.023211194, -0.028653989, 0.024858678, 0.016218547, -0.021331845, -0.041467737, 0.042956576, 0.016950762, 0.00016636902, 0.03922228, 0.011904585, 0

In [4]:
index_host = indexRes.index.host

In [7]:
def upsert_embedding(values:list[list[float]]):
    index = pinecone.Index(host=index_host)
    data = []
    for i in values:
        data.append({
            "id": "uid",
            "values": i,
            "metadata": {"type": "mildly depressed"}
        })
    
    res = index.upsert(
        vectors=data,
        namespace="bezzie-testing"
    )
    return res
values = [embd.data[0].embedding]
res = upsert_embedding(values)
print(res)

{'upserted_count': 1}
